# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 23 2022 1:29PM,240316,10,0085798166,ISDIN Corporation,Released
1,Jun 23 2022 1:29PM,240316,10,0085798162,ISDIN Corporation,Released
2,Jun 23 2022 1:25PM,240315,10,8183346,"Methapharm, Inc.",Released
3,Jun 23 2022 1:24PM,240314,10,8183343,"Methapharm, Inc.",Released
4,Jun 23 2022 1:23PM,240313,10,0085798164,ISDIN Corporation,Released
5,Jun 23 2022 1:23PM,240313,10,0085798174,ISDIN Corporation,Released
6,Jun 23 2022 1:23PM,240313,10,0085798177,ISDIN Corporation,Released
7,Jun 23 2022 1:23PM,240313,10,0085798180,ISDIN Corporation,Released
8,Jun 23 2022 1:23PM,240313,10,0085798199,ISDIN Corporation,Released
9,Jun 23 2022 1:23PM,240313,10,0085798200,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,240312,Released,1
19,240313,Released,16
20,240314,Released,1
21,240315,Released,1
22,240316,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240312,NaN,NaN,1.0
240313,NaN,NaN,16.0
240314,NaN,NaN,1.0
240315,NaN,NaN,1.0
240316,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240194,0.0,0.0,1.0
240235,0.0,0.0,1.0
240237,0.0,0.0,1.0
240239,0.0,0.0,1.0
240241,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240194,0,0,1
240235,0,0,1
240237,0,0,1
240239,0,0,1
240241,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240194,0,0,1
1,240235,0,0,1
2,240237,0,0,1
3,240239,0,0,1
4,240241,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240194,,,1
1,240235,,,1
2,240237,,,1
3,240239,,,1
4,240241,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 23 2022 1:29PM,240316,10,ISDIN Corporation
2,Jun 23 2022 1:25PM,240315,10,"Methapharm, Inc."
3,Jun 23 2022 1:24PM,240314,10,"Methapharm, Inc."
4,Jun 23 2022 1:23PM,240313,10,ISDIN Corporation
20,Jun 23 2022 1:21PM,240312,10,"Methapharm, Inc."
21,Jun 23 2022 1:04PM,240309,20,"ACI Healthcare USA, Inc."
28,Jun 23 2022 12:52PM,240307,12,"Uniderm USA, Inc"
31,Jun 23 2022 12:05PM,240298,18,"Emersa Waterbox, LLC"
32,Jun 23 2022 11:40AM,240295,21,"NBTY Global, Inc."
33,Jun 23 2022 11:11AM,240291,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 23 2022 1:29PM,240316,10,ISDIN Corporation,,,2
1,Jun 23 2022 1:25PM,240315,10,"Methapharm, Inc.",,,1
2,Jun 23 2022 1:24PM,240314,10,"Methapharm, Inc.",,,1
3,Jun 23 2022 1:23PM,240313,10,ISDIN Corporation,,,16
4,Jun 23 2022 1:21PM,240312,10,"Methapharm, Inc.",,,1
5,Jun 23 2022 1:04PM,240309,20,"ACI Healthcare USA, Inc.",,1,6
6,Jun 23 2022 12:52PM,240307,12,"Uniderm USA, Inc",,,3
7,Jun 23 2022 12:05PM,240298,18,"Emersa Waterbox, LLC",,,1
8,Jun 23 2022 11:40AM,240295,21,"NBTY Global, Inc.",,,1
9,Jun 23 2022 11:11AM,240291,10,ISDIN Corporation,,29,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 1:29PM,240316,10,ISDIN Corporation,2,,
1,Jun 23 2022 1:25PM,240315,10,"Methapharm, Inc.",1,,
2,Jun 23 2022 1:24PM,240314,10,"Methapharm, Inc.",1,,
3,Jun 23 2022 1:23PM,240313,10,ISDIN Corporation,16,,
4,Jun 23 2022 1:21PM,240312,10,"Methapharm, Inc.",1,,
5,Jun 23 2022 1:04PM,240309,20,"ACI Healthcare USA, Inc.",6,1,
6,Jun 23 2022 12:52PM,240307,12,"Uniderm USA, Inc",3,,
7,Jun 23 2022 12:05PM,240298,18,"Emersa Waterbox, LLC",1,,
8,Jun 23 2022 11:40AM,240295,21,"NBTY Global, Inc.",1,,
9,Jun 23 2022 11:11AM,240291,10,ISDIN Corporation,,29,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 1:29PM,240316,10,ISDIN Corporation,2,,
1,Jun 23 2022 1:25PM,240315,10,"Methapharm, Inc.",1,,
2,Jun 23 2022 1:24PM,240314,10,"Methapharm, Inc.",1,,
3,Jun 23 2022 1:23PM,240313,10,ISDIN Corporation,16,,
4,Jun 23 2022 1:21PM,240312,10,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 1:29PM,240316,10,ISDIN Corporation,2.0,NaN,NaN
1,Jun 23 2022 1:25PM,240315,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Jun 23 2022 1:24PM,240314,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Jun 23 2022 1:23PM,240313,10,ISDIN Corporation,16.0,NaN,NaN
4,Jun 23 2022 1:21PM,240312,10,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 1:29PM,240316,10,ISDIN Corporation,2.0,0.0,0.0
1,Jun 23 2022 1:25PM,240315,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Jun 23 2022 1:24PM,240314,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Jun 23 2022 1:23PM,240313,10,ISDIN Corporation,16.0,0.0,0.0
4,Jun 23 2022 1:21PM,240312,10,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1922425,46.0,34.0,9.0
12,480590,3.0,1.0,0.0
16,960940,4.0,0.0,0.0
18,240298,1.0,0.0,0.0
19,240285,1.0,0.0,0.0
20,961052,10.0,1.0,0.0
21,240295,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1922425,46.0,34.0,9.0
1,12,480590,3.0,1.0,0.0
2,16,960940,4.0,0.0,0.0
3,18,240298,1.0,0.0,0.0
4,19,240285,1.0,0.0,0.0
5,20,961052,10.0,1.0,0.0
6,21,240295,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,46.0,34.0,9.0
1,12,3.0,1.0,0.0
2,16,4.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,1.0,0.0,0.0
5,20,10.0,1.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,46.0
1,12,Released,3.0
2,16,Released,4.0
3,18,Released,1.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21
Status,,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,34.0,1.0,0.0,0.0,0.0,1.0,0.0
Released,46.0,3.0,4.0,1.0,1.0,10.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,1.0,0.0,0.0,0.0,1.0,0.0
2,Released,46.0,3.0,4.0,1.0,1.0,10.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,1.0,0.0,0.0,0.0,1.0,0.0
2,Released,46.0,3.0,4.0,1.0,1.0,10.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()